In [131]:
%load_ext autoreload
%autoreload 1

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings("ignore")

In [41]:
import json
from pandas.io.json import json_normalize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import (roc_auc_score, precision_score, recall_score, 
                             confusion_matrix, accuracy_score, f1_score,
                             classification_report)
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler, MaxAbsScaler
import numpy as np
from nltk.corpus import stopwords
import xgboost as xgb
from pathlib import Path
from tokenize_uk.tokenize_uk import tokenize_words
from operator import itemgetter
from scipy import stats

In [133]:
%aimport config
from config import params

In [35]:
def calc_metrics(y_test, pred, proba=None, labels=["1", "2", "3", "4", "5"], print_=True,
                 average="macro", report=True):
    output = {}
    if proba is not None:
        roc_auc = roc_auc_score(y_test, proba)
        output["AUC"] = roc_auc
    output["Recall"] = recall_score(y_test, pred, average=average)
    output["Precision"] = precision_score(y_test, pred, average=average)
    output["F1"] = f1_score(y_test, pred, average=average)
    output["accuracy"] = accuracy_score(y_test, pred)
    if labels is not None:
        index = labels
        columns = ["pred_" + el for el in index]
    else:
        columns = None
        index = None
    output["conf_matrix"] = pd.DataFrame(confusion_matrix(y_test, pred), 
                                         columns=columns, index=index)
    if print_:
        for key, value in output.items():
            if "matrix" in key:
                print(value)
            else:
                print(f"{key}: {value:0.3f}")
    if report:
        output["report"] = classification_report(y_test, pred, labels)
        print(output["report"])
    return output

In [470]:
with open("stop_words.txt") as file:
    stop = file.readlines()
stop = [el.strip() for el in stop]

In [6]:
tone = pd.read_csv("tone-dict.tsv", sep='\t', header=None, names=["word", "sentiment"])
tone["word"] = tone["word"].str.lower()
tone_map = tone.set_index("word").to_dict()["sentiment"]

In [471]:
data = []
for file in Path().glob("smart*.json"):
    with open(file, "r") as f_in:
        data.extend(json.load(f_in))

In [448]:
df = json_normalize(data, record_path="reviews", meta=["path", "price", "title"])

In [457]:
df["review"] = df.apply(lambda x: x['text'] + " " + x['pros']+ " " + x['cons'], axis=1)
#frq = df.groupby("stars")["text"].count()
#df["weights"] = df["stars"].map({"1": 1, "2": 1, "3": 1, "4": 5, "5": 3})

In [458]:
df.head(1)
df["stars"].describe()

,author,cons,date,link,pros,stars,text,path,price,title,review
0,балацька лариса,До телефона треба зразу брати і захисник скло ...,2018-3-24,https://rozetka.com.ua/zte_blade_v8_gray/p2573...,Камера і батарея,5,"Купили телефон в подарунок дітям, зразу два зо...","[Интернет-супермаркет №, Смартфоны, ТВ и элект...",4499,ZTE Blade V8 Gray,"Купили телефон в подарунок дітям, зразу два зо..."


count    3797.000000
mean        4.359494
std         1.004769
min         1.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: stars, dtype: float64

#### Distribution of classes

In [451]:
s = df.stars.value_counts()
pd.concat([s, s / s.sum()], axis=1)

,stars,stars
5,2293,0.603898
4,988,0.260205
3,243,0.063998
1,139,0.036608
2,134,0.035291


So, we deal with an imbalanced multiclass classification problem

In [65]:
# Analyze cons
df["cons"].value_counts().head(10)
patt = "не має|не знайш|немає|не вияв|відсутн|нема|--"
cond = ((df["cons"].str.lower().str.contains(patt, regex=True)) | (df["cons"]==""))
df.loc[~cond, "stars"].mean()
df.loc[cond, "stars"].mean()

               700
Немає           92
немає           85
-               34
не виявлено     31
Не виявлено     20
нема            20
Відсутні        19
Не виявила      16
Нема            15
Name: cons, dtype: int64

4.088316467341306

4.700239808153477

In [67]:
# Analyze pros
df["pros"].value_counts().head(10)
patt = "ціна|якість"
cond1 = (df["pros"].str.lower().str.contains(patt, regex=True)) | (df["pros"].str.len() < 10)
cond2 = df["pros"]==""
cond = cond2
df.loc[~cond, "stars"].mean()
df.loc[cond, "stars"].mean()

                  626
Ціна               27
ціна               13
Дизайн              7
дешевий             6
Ціна-якість         5
Дешевий             5
Тиха                5
Ціна та якість      5
немає               5
Name: pros, dtype: int64

4.465476749647722

4.436102236421725

#### Train / test split

Use 20% of data for testing and stratify according to the target variable

In [459]:
y = df["stars"]
X = df.loc[:, ~df.columns.isin(["stars"])]
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,
                                                    stratify=y)
print(f"Num. of train: {len(X_train)}, Num. of test: {len(X_test)}")

Num. of train: 3037, Num. of test: 760


#### Baseline

Let's evaluate performance of the very simple baseline prediction algorithm - <b>everything is a 5 star review (just take a majority class)</b>

In [453]:
base_pred = np.full(y_test.shape, 5)
base_metrics = calc_metrics(y_test, base_pred, proba=None, average="weighted")

Recall: 0.604
Precision: 0.365
F1: 0.455
accuracy: 0.604
   pred_1  pred_2  pred_3  pred_4  pred_5
1       0       0       0       0      28
2       0       0       0       0      27
3       0       0       0       0      48
4       0       0       0       0     198
5       0       0       0       0     459
             precision    recall  f1-score   support

          1       0.00      0.00      0.00        28
          2       0.00      0.00      0.00        27
          3       0.00      0.00      0.00        48
          4       0.00      0.00      0.00       198
          5       0.60      1.00      0.75       459

avg / total       0.36      0.60      0.45       760



#### TF - IDF & other fesatures

In [439]:
def validate_array(x):
    if len(x) > 0:
        return x
    return np.array([0])

In [467]:
def f(x):
    x = validate_array(np.array(x))
    nonzero = np.nonzero(x)[0].shape[0]
    n_pos = (x > 0).sum()
    n_neg = (x < 0).sum()
    pos = validate_array(x[x > 0])
    neg = validate_array(x[x < 0])
    max_pos = np.max(pos)
    min_pos = np.min(pos)
    mean_pos = np.mean(pos)
    std_pos = np.std(pos)
    sum_pos = np.sum(pos)
    max_neg = np.max(neg)
    min_neg = np.min(neg)
    mean_neg = np.mean(neg)
    std_neg = np.std(neg)
    sum_neg = np.sum(neg)
    return pd.Series({"nonzero": nonzero, 
                      "nonzero_ratio": nonzero / len(x) if len(x)>0 else 0,
                      "n_pos": n_pos,
                      "n_pos_ratio": n_pos / nonzero if nonzero>0 else 0,
                      "n_neg": n_neg,
                      "n_neg_ratio": n_neg / nonzero if nonzero>0 else 0,
                      "mean": np.mean(x),
                      "max": np.max(x),
                      "min": np.min(x),
                      "std": np.std(x),
                      "sum": np.sum(x),
                      "max_pos": max_pos,
                      "min_pos": min_pos,
                      "mean_pos": mean_pos,
                      "std_pos": std_pos,
                      "sum_pos": sum_pos,
                      "max_neg": max_neg,
                      "min_neg": min_neg,
                      "mean_neg": mean_neg,
                      "std_neg": std_neg,
                      "sum_neg": sum_neg 
                       })

In [340]:
def condition_cons(df, var="cons", patt="не має|не знайш|немає|не вияв|відсутн|нема|--"):
    cond = ((df[var].str.lower().str.contains(patt, regex=True)) | (df[var]==""))
    return cond

In [469]:
def add_tone(X, tone_map=tone_map, var="review"):
    temp = X[var].map(lambda x: [tone_map.get(word.lower(), 0) for word in tokenize_words(x)])
    return temp.apply(f)

In [58]:
def apply_transformer(train, test, transformer):
    if transformer is not None:
        train = transformer.fit_transform(train)
        test = transformer.transform(test)
        return train, test
    return train, test

In [342]:
def build_features(X_train, X_test, transformer=None, var="review", features=None,
                   vectorizer=None):
    f_train = []
    f_test = []
    for feature in features:
        if feature == "tfidf":
            train = vectorizer.fit_transform(X_train[var]).toarray()
            test = vectorizer.transform(X_test[var]).toarray()
            f_train.append(train)
            f_test.append(test)
        elif feature == "sentiment":
            train = add_tone(X_train, var=var).values
            test = add_tone(X_test, var=var).values
            train, test = apply_transformer(train, test, transformer)
            f_train.append(train)
            f_test.append(test)
        elif feature == "len":
            train = X_train[["text", "pros", "cons"]].applymap(len).values
            test = X_test[["text", "pros", "cons"]].applymap(len).values
            train, test = apply_transformer(train, test, transformer)
            f_train.append(train)
            f_test.append(test)
        elif feature == "is_cons":
            train = condition_cons(X_train).astype(int)[:, np.newaxis]
            test = condition_cons(X_test).astype(int)[:, np.newaxis]
            f_train.append(train)
            f_test.append(test)
    return np.concatenate((f_train), axis=1), np.concatenate((f_test), axis=1)

In [221]:
def fit_nb(train, y_train, test, y_test, alpha=2.5, priors=None):
    clf = MultinomialNB(alpha=alpha, class_prior=priors)
    clf.fit(train, y_train)
    pred = clf.predict(test)
    proba = clf.predict_proba(test)
    metrics = calc_metrics(y_test, pred, proba=None, average="weighted")
    return pred, proba, metrics

In [60]:
quant = QuantileTransformer(n_quantiles=10, output_distribution="uniform")
minmax = MinMaxScaler()
std = StandardScaler()
maxabs = MaxAbsScaler()
transformer = minmax

Try only TF-IDF (with stop words if we use analyzer=word)

In [473]:
tf_params = {"lowercase": 1,
             "analyzer": "word",
             "stop_words": stop,
             "ngram_range": (1, 1),
             "min_df": 1,
             "max_df": 1.0,
             "preprocessor": None,
             "max_features": 3500*1 or None,
             "norm": 'l2'*0,
             "use_idf": 0,
             "smooth_idf": 0,
             "sublinear_tf": 0, 
             "tokenizer": None#tokenize_words
             }
var = "review"
features = [
            "tfidf",
            #"sentiment", 
            #"len", 
            #"is_cons"
            ]
vectorizer = TfidfVectorizer(**tf_params)
train, test = build_features(X_train, X_test, features=features, vectorizer=vectorizer,
                            transformer=transformer, var=var)
print(f"Features: {train.shape[1]}")

Features: 3500


In [474]:
pred, proba, metrics = fit_nb(train, y_train, test, y_test, alpha=2.6)

Recall: 0.634
Precision: 0.561
F1: 0.586
accuracy: 0.634
   pred_1  pred_2  pred_3  pred_4  pred_5
1       2       0       2      13      11
2       1       0       3      13      10
3       2       0       1      27      18
4       0       0       4      73     121
5       0       1       1      51     406
             precision    recall  f1-score   support

          1       0.40      0.07      0.12        28
          2       0.00      0.00      0.00        27
          3       0.09      0.02      0.03        48
          4       0.41      0.37      0.39       198
          5       0.72      0.88      0.79       459

avg / total       0.56      0.63      0.59       760



##### Let's add sentiments

In [477]:
features = [
            "tfidf",
            "sentiment", 
            #"len", 
            #"is_cons"
            ]
train, test = build_features(X_train, X_test, features=features, vectorizer=vectorizer,
                            transformer=transformer, var=var)
print(f"Features: {train.shape[1]}")

Features: 3521


In [478]:
pred, proba, metrics = fit_nb(train, y_train, test, y_test, alpha=2.6)

Recall: 0.646
Precision: 0.558
F1: 0.586
accuracy: 0.646
   pred_1  pred_2  pred_3  pred_4  pred_5
1       1       0       2      11      14
2       1       0       2      15       9
3       0       0       0      30      18
4       0       0       0      68     130
5       0       0       0      37     422
             precision    recall  f1-score   support

          1       0.50      0.04      0.07        28
          2       0.00      0.00      0.00        27
          3       0.00      0.00      0.00        48
          4       0.42      0.34      0.38       198
          5       0.71      0.92      0.80       459

avg / total       0.56      0.65      0.59       760



#### Fit XGBoost

In [380]:
params['num_class'] = df["stars"].nunique()

In [377]:
dtrain = xgb.DMatrix(train, y_train-1)#, weight=X_train["weights"])
dtest = xgb.DMatrix(test, y_test-1)#, weight=X_test["weights"])
eval_set = [(dtrain, "train"), (dtest, "eval")]

In [381]:
model = xgb.train(dtrain=dtrain, num_boost_round=params.get("n_estimators"), 
                  early_stopping_rounds=params.get("early_stopping_rounds"), 
                  params=params, evals=eval_set, verbose_eval=50)

[0]	train-mlogloss:1.50738	train-merror:0.325771	eval-mlogloss:1.50779	eval-merror:0.333938
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 50 rounds.
[50]	train-mlogloss:0.847278	train-merror:0.323503	eval-mlogloss:0.87719	eval-merror:0.319419
[100]	train-mlogloss:0.825416	train-merror:0.317151	eval-mlogloss:0.865001	eval-merror:0.317604
Stopping. Best iteration:
[52]	train-mlogloss:0.845629	train-merror:0.323049	eval-mlogloss:0.875408	eval-merror:0.315789



In [382]:
pred_xgb = (model.predict(dtest)+1).astype(int)
print(f"Best score (accuracy): {1-model.best_score}")
xgb_metrics = calc_metrics(y_test, pred_xgb, average="weighted")

Best score (accuracy): 0.684211
Recall: 0.679
Precision: 0.564
F1: 0.605
accuracy: 0.679
   pred_1  pred_2  pred_3  pred_4  pred_5
1       0       0       0      10       9
2       0       0       0       5       8
3       0       0       0       7      24
4       0       0       0      26      96
5       0       0       0      18     348
             precision    recall  f1-score   support

          1       0.00      0.00      0.00        19
          2       0.00      0.00      0.00        13
          3       0.00      0.00      0.00        31
          4       0.39      0.21      0.28       122
          5       0.72      0.95      0.82       366

avg / total       0.56      0.68      0.60       551

